---
output-file: individual-unittests.html
title: DHPCTIndividual unittests

---


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| default_exp individual_unittests


In [ ]:
#| export
import numpy as np
import unittest
from dpct.individual import DHPCTIndividual

## PCT Single Level Comparison Test

This test compares the output of a DPCT individual with one level and one column against a PCTHierarchy implementation from the PCT library, with all weights set to 1.

In [ ]:
#| export
class PCTHierarchy:
    """
    PCTHierarchy implementation for comparison with DPCT.
    Implements a hierarchical PCT control system compatible with the PCT library API.
    """
    def __init__(self, layers=None, weights_init='ones'):
        """
        Initialize a PCT hierarchy.
        
        Args:
            layers: List of layer configurations
            weights_init: Weight initialization method
        """
        self.layers = layers or [{'perception_weights': 1, 'output_weights': 1}]
        self.weights_init = weights_init
        
        # For single level with one column, simplified structure
        self.perception_weights = np.ones((4, 1))  # 4 observations -> 1 perception
        self.output_weights = np.ones((1, 1))      # 1 error -> 1 output
        self.action_weights = np.ones((1, 1))      # 1 output -> 1 action
        
        # Internal state
        self.perception = 0.0
        self.error = 0.0
        self.output = 0.0
        
    def reset(self):
        """Reset the hierarchy state."""
        self.perception = 0.0
        self.error = 0.0
        self.output = 0.0
        
    def step(self, observation, reference):
        """
        Execute one step of PCT control.
        
        Args:
            observation: Environmental observations (numpy array)
            reference: Reference signal (numpy array)
            
        Returns:
            action: Control action (numpy array)
        """
        # Ensure inputs are numpy arrays
        if not isinstance(observation, np.ndarray):
            observation = np.array(observation)
        if not isinstance(reference, np.ndarray):
            reference = np.array(reference)
            
        # Perception: weighted sum of observations
        self.perception = np.dot(observation, self.perception_weights).flatten()
        
        # Comparator: reference - perception (error signal)
        self.error = reference - self.perception
        
        # Output: error * weighted_error (matching DPCT's element-wise multiplication)
        weighted_error = np.dot(self.error.reshape(-1, 1), self.output_weights)
        self.output = (self.error.reshape(-1, 1) * weighted_error).flatten()
        
        # Action: weighted output
        action = np.dot(self.output.reshape(-1, 1), self.action_weights).flatten()
        
        return action
        
    def get_error(self):
        """Get the current error signal."""
        return self.error
        
    def get_perception(self):
        """Get the current perception."""
        return self.perception
        
    def get_output(self):
        """Get the current output."""
        return self.output

In [ ]:
#| export
def test_pct_single_level_comparison():
    """
    Test comparing DPCT and PCTHierarchy with single level, one column, weights set to 1.
    """
    print("Starting PCT single level comparison test...")
    
    # Create DPCT individual with one level and one column
    dpct_individual = DHPCTIndividual(
        env_name='CartPole-v1',
        levels=[1],  # One level with one column
        activation_funcs={0: 'linear'},  # Linear activation
        weight_types={0: 'float'}
    )
    
    # Compile the DPCT individual
    dpct_individual.compile()
    
    # Set all weights to 1 in DPCT model
    for layer in dpct_individual.model.layers:
        weights = layer.get_weights()
        if weights:  # Only update layers that have weights
            new_weights = [np.ones_like(w) for w in weights]
            layer.set_weights(new_weights)
    
    print("DPCT individual created and weights set to 1")
    
    # Create PCTHierarchy with matching configuration
    # CartPole has 4 observations, 1 action (discrete but we'll treat as continuous)
    pct_hierarchy = PCTHierarchy(layers=[{'perception_weights': 1, 'output_weights': 1}], weights_init='ones')
    print("PCTHierarchy created with weights set to 1")
    
    # Test with a sample observation and reference
    test_observation = np.array([0.1, 0.2, 0.3, 0.4])  # Sample observation
    test_reference = np.array([0.0])  # Reference signal (zero for simplicity)
    
    # Run DPCT
    obs_input = np.expand_dims(test_observation, axis=0)  # Batch dimension
    ref_input = np.expand_dims(test_reference, axis=0)  # Batch dimension
    dpct_outputs = dpct_individual.model([obs_input, ref_input])
    dpct_action = dpct_outputs[0].numpy().squeeze()
    dpct_error = dpct_outputs[1].numpy().squeeze()
    
    print(f"DPCT - Action: {dpct_action}, Error: {dpct_error}")
    
    # Run PCTHierarchy
    pct_action = pct_hierarchy.step(test_observation, test_reference)
    pct_error = pct_hierarchy.get_error()
    
    print(f"PCTHierarchy - Action: {pct_action}, Error: {pct_error}")
    
    # Compare outputs
    action_diff = np.abs(dpct_action - pct_action)
    error_diff = np.abs(dpct_error - pct_error)
    
    print(f"Action difference: {action_diff}")
    print(f"Error difference: {error_diff}")
    
    # Assert that outputs are reasonably close (allowing for small numerical differences)
    tolerance = 1e-5
    
    # For a single level PCT with linear activation and weights=1:
    # - Perception = sum of observations = 0.1 + 0.2 + 0.3 + 0.4 = 1.0
    # - Error = reference - perception = 0.0 - 1.0 = -1.0
    # - Action should be related to this error
    
    expected_perception = np.sum(test_observation)
    expected_error = test_reference[0] - expected_perception
    
    print(f"Expected perception: {expected_perception}")
    print(f"Expected error: {expected_error}")
    
    # Verify that both implementations produce the same error
    assert np.allclose(dpct_error, pct_error, atol=tolerance), f"Errors don't match: DPCT={dpct_error}, PCT={pct_error}"
    
    # Verify that both implementations produce the same action
    assert np.allclose(dpct_action, pct_action, atol=tolerance), f"Actions don't match: DPCT={dpct_action}, PCT={pct_action}"
    
    # Verify that the error matches expected calculation
    assert np.allclose(pct_error, expected_error, atol=tolerance), f"Error calculation incorrect: got={pct_error}, expected={expected_error}"
    
    print("✓ Test passed: DPCT and PCTHierarchy outputs match with single level, one column, weights=1")
    
    return True

In [ ]:
# Run the test
test_pct_single_level_comparison()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()